In [1]:
!python --version
!pip install --upgrade pip
!pip install numpy==1.25
!pip install fairlearn==0.9.0
!pip install plotly==5.24.1
!pip install nbformat==5.10.4
!pip install ipykernel==6.29.5

Python 3.10.11
  Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)


ERROR: To modify pip, please run the following command:
C:\Users\dewey\Desktop\L3\Fairness\Projet\myenv\Scripts\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as plt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import fairlearn
from statsmodels.graphics.mosaicplot import mosaic
from scipy.stats import spearmanr, pearsonr
from statsmodels.graphics.mosaicplot import mosaic
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", append=True, category=UserWarning)

# Datasets
from aif360.datasets import MEPSDataset19
from aif360.explainers import MetricTextExplainer

# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.algorithms.preprocessing import Reweighing

np.__version__, fairlearn.__version__


('1.25.0', '0.9.0')

# Introduction

Notre  jeu de données est un extrait du jeu de données "Chest X ray NIH 14 Dataset" qui est un Dataset d’image de rayons X, fourni avec de nombreuses informations sur les
personnes (notamment age, genre, maladies). Notre extrait contient 55552 lignes correspondant à des diagnostics, avec possiblement des personnes présentes dans plusieurs cas. Chaque donnée possède 12 caractéristiques concernant des informations et précisions médicales, ainsi que des informations personnels sur les patients.
Les principales caractéristiques de notre jeu de données sont :
- Image Index : Une image rayon X
- Finding labels : Libellé(s) des diagnostics
- Follow-up : Suivis du médecin
- Patient ID : Identifiant unique du patient
- Patient Age : Age du patient
- Patient Gender : Le gendre du patient
- View Position	: Position de la prise de vue
- OriginalImage Width : largeur de l'image rayon X
- OriginalImage Height : hauteur de l'image rayon X
- OriginalImagePixelSpacing x : Distance physique représentée par un pixel sur l’axe horizontal 
- OriginalImagePixelSpacing y : Distance physique représentée par un pixel sur l’axe vertical
- Unnamed: 11 : Une colonne sans informations ( et donc sans réels interêt )

    L'objectif du projet est de trouver des biais dans notre jeu de données, grâce à l'analyse
bivariée et univariée dans un 1er temps, puis de les corriger grâce aux méthodes de pre-processing vues en cours et que l'on appliquera dans la suite du projet.

# 1 - Chargement du Jeu de données

In [34]:
df = pd.read_csv("data/WANE_ISMAEL.csv")
print(df.shape)
df

(55552, 12)


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
55547,00030798_000.png,No Finding,0,30798,30,M,PA,2500,2048,0.171,0.171,NaN
55548,00030801_000.png,No Finding,0,30801,39,M,PA,2500,2048,0.168,0.168,NaN
55549,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168,NaN
55550,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168,NaN


In [35]:
df.dtypes

Image Index                     object
Finding Labels                  object
Follow-up #                      int64
Patient ID                       int64
Patient Age                      int64
Patient Gender                  object
View Position                   object
OriginalImage[Width              int64
Height]                          int64
OriginalImagePixelSpacing[x    float64
y]                             float64
Unnamed: 11                    float64
dtype: object

# 2 - Nettoyage/Préparation des données

In [36]:
# On vérifie si il y'a des valeurs Nan dans notre dataframe
df_num = df.select_dtypes(include=['number'])
for column in df_num.columns:
  print(column, df[column].isnull().values.any())

Follow-up # False
Patient ID False
Patient Age False
OriginalImage[Width False
Height] False
OriginalImagePixelSpacing[x False
y] False
Unnamed: 11 True


In [37]:
# On observe que presque toutes nos colonnes numériques ne contiennent pas de valeurs manquantes NaN,
# à part la colonne Unnamed: 11.
# Reste à savoir si toutes ses valeurs sont des NaN

vlr = df['Unnamed: 11'].nunique(dropna=False)
vlr


1

In [38]:
# Il n'y a qu'une unique valeur dans cette colonne, qui est NaN, donc cette colonne ne porte aucune information.
# Nous pouvons la supprimer

df_clean = df.drop(columns="Unnamed: 11")
df_clean

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143
...,...,...,...,...,...,...,...,...,...,...,...
55547,00030798_000.png,No Finding,0,30798,30,M,PA,2500,2048,0.171,0.171
55548,00030801_000.png,No Finding,0,30801,39,M,PA,2500,2048,0.168,0.168
55549,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168
55550,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168


In [39]:
#Pour une meilleure visibilité, on va renommer certaines colonnes
df_clean = df_clean.rename(columns={
    "OriginalImage[Width": "Original_Image_Width",
    "Height]": "Original_Image_Height",
    "OriginalImagePixelSpacing[x": "PixelSpacingX",
    "y]": "PixelSpacingY"
})
df_clean

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,Original_Image_Width,Original_Image_Height,PixelSpacingX,PixelSpacingY
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143
...,...,...,...,...,...,...,...,...,...,...,...
55547,00030798_000.png,No Finding,0,30798,30,M,PA,2500,2048,0.171,0.171
55548,00030801_000.png,No Finding,0,30801,39,M,PA,2500,2048,0.168,0.168
55549,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168
55550,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168


# 3 - Analyse des données

Nous allons ensuite effectuer l'analyse descriptive de nos données, en commençant par leur analyse univariée et en terminant sur leur analyse bivariée.

# Analyse univariée

In [40]:
# Conversion de la colonne "Patient Age" en numérique
df_clean['Patient Age'] = pd.to_numeric(df_clean['Patient Age'], errors='coerce')

In [41]:
# Histogramme de la distribution des âges
plt.figure(figsize=(10, 5))
sns.histplot(df_clean["Patient Age"], bins=30, kde=True, color="skyblue")
plt.title("Distribution des âges des patients")
plt.xlabel("Âge")
plt.ylabel("Fréquence")
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()


In [42]:
# Ce qui est mis en avant par l'histogramme est encore plus visible par le schéma suivant et sa
#description

plt.figure(figsize=(10, 5))
sns.boxplot(x=df_clean["Patient Age"])
plt.title("Distribution des âges des patients")
plt.show()
print("Statistiques sur les âges :")
display(df_clean["Patient Age"].describe())

Statistiques sur les âges :


count    55552.000000
mean        46.913270
std         16.589761
min          1.000000
25%         35.000000
50%         48.000000
75%         59.000000
max        414.000000
Name: Patient Age, dtype: float64

D'après les statistiques réalisés sur l'attribut "âge", on voit que l'âge moyen est de 46 ans, avec un âge minimum de 1 ans et un maximum de 414 ans, ce qui assez paradoxale.
On remarque aussi un 1er biais, c'est à dire, des individus dont l'âge est iréaliste, on en trouve 3 si on part du principe que l'age maximal est de 100 ans. On observe 3 personnes avec un âge irrealiste.

In [43]:
df_filtered = df_clean[df_clean["Patient Age"] > 100]
df_filtered
#414, 148, 150 très très loin de la durée de vie record qui est de de 122 ans.

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,Original_Image_Width,Original_Image_Height,PixelSpacingX,PixelSpacingY
23236,00011973_002.png,Edema,2,11973,414,M,AP,3056,2544,0.139,0.139
23984,00012238_010.png,No Finding,10,12238,148,M,PA,2992,2991,0.143,0.143
29342,00014520_026.png,Infiltration|Mass,26,14520,150,M,PA,2992,2991,0.143,0.143


In [44]:
def proportion_par_decennie():
    df["Decennie_Age"] = (df["Patient Age"] // 10) * 10  # Regrouper les âges par dizaines
    age_counts = df["Decennie_Age"].value_counts(normalize=True).sort_index() * 100  # Pourcentage
    
    print("\nProportion de personnes par décennie d'âge:")
    print(age_counts)
    
    plt.figure(figsize=(8,5))
    sns.barplot(x=age_counts.index, y=age_counts.values, palette="Blues_r")
    plt.xlabel("Décennie d'âge")
    plt.ylabel("Proportion (%)")
    plt.title("Répartition des âges par décennie")
    plt.xticks(rotation=45)
    plt.show()
    
proportion_par_decennie()


Proportion de personnes par décennie d'âge:
Decennie_Age
0       1.146673
10      4.685700
20     11.439732
30     14.438724
40     20.605919
50     23.919931
60     16.593462
70      6.073589
80      1.002664
90      0.088206
140     0.001800
150     0.001800
410     0.001800
Name: proportion, dtype: float64


Un autre biais pourrait être observé au niveau de la répartition de l'age, on observe si l'on classe par categorie par décennie, une nette domination de personnes entre 45 et 60 ans. Cela pourrait constituer aussi un biais associé à une autre feature.

In [45]:
# Analyse de la répartition des genres
plt.figure(figsize=(6, 4))
sns.countplot(x=df_clean["Patient Gender"], palette = "Set2")
plt.title("Répartition des genres")
plt.xlabel("Genre")
plt.ylabel("Nombre d'individus")
plt.show()
print("Nombre d'individus par genre :")
display(df_clean["Patient Gender"].value_counts())

Nombre d'individus par genre :


Patient Gender
M    30838
F    24714
Name: count, dtype: int64

On observe donc qu'il y a beaucoup de moins de femmes dans notre jeu de données, comme le montre la description du jeu de données et le graphique réalisé en fonction de l'attribut "Patient Gender".
Cela pourrait constituer un autre biais à corriger dans notre jeu de données. L'echantillonage serait une bonne solution cependant nous avons de meilleures méthodes dans le cours.

In [46]:
df_clean["Finding Labels"].value_counts().nunique()

93

On a 93 valeurs différentes possibles dans notre colonne "Finding Labels". On va observer leur repartion dans nos données. Pour eviter de surcharger notre affichage, on va rassembler les valeurs de "Finding Labels" trés peu representées

In [47]:

# Compter les diagnostics
diagnosis_counts = df_clean["Finding Labels"].value_counts()

# Séparer les 9 plus fréquents et sommer le reste, pour une meilleure visualisation
top_diagnoses = diagnosis_counts.nlargest(9)
others_count = diagnosis_counts.iloc[10:].sum()

# Créer une nouvelle série avec "Autres"
diagnosis_pie = pd.concat([top_diagnoses, pd.Series(others_count, index=["Autres"])])


# Affichage du camembert
plt.figure(figsize=(10, 6))
plt.pie(
    diagnosis_pie, 
    labels=diagnosis_pie.index, 
    autopct='%1.1f%%', 
    startangle=140, 
    colors=sns.color_palette("Set3")
)
plt.title("Répartition des 10 diagnostics les plus fréquents (+ Autres)")
plt.show()


Ici, on remarque que la valeur "No Finding" est la valeur présente en très large majorité dans la colonne "Finding Labels"(54.0%), suivi largement après par "Infiltration" avec seulement 8.6%. Cela peut également constitué un biais pour la suite du projet. Dans le cas d'un modèle d'apprentissage, on pourrait observer des performances réduites sur des pathologies moins fréquentes.
On pourrait partir sur un principe pour regler ce biais, on pourrait essayer d'équilibrer nos données afin d'avoir 50% de No finding et 50% de personnes diagnostiquées.

Seul, il n'y a pas d'interêt à faire l'analyse univarié de la colonne "Follow-up #", car la valeur derrière de suivis du patient aurait plus de sens associée à une autre colonne ( etudier une correlation par exemple etc...)

In [48]:
df_clean["View Position"].value_counts()

View Position
PA    32912
AP    22640
Name: count, dtype: int64

On remarque aussi qu'il y a beaucoup plus d'images prises en PA que d'images prises en AP. Cela pourrait constituer un biais. 
Compte tenu de la consigne, nous ne nous interesserons qu'aux métadonnées et non aux images et ses caractéristiques.

# Analyse bivariée

On va commencer par faire l'analyse bivariée de variables quantitatives. ( On ne s'interesse qu'aux métadonnées et pas aux images )

In [49]:
#Chargement des variables quantitatives
qualitative_vars = df_clean[['Finding Labels', 'Patient Gender']].columns.tolist()
#Chargement des variables qualitatives
quantitative_vars = df_clean[['Patient ID', 'Patient Age', 'Follow-up #']].columns.tolist()

In [50]:
#Pour deux variables quantitatives

# Création de nuages de points pour toutes les paires de variables quantitatives
sns.pairplot(df_clean[quantitative_vars], diag_kind='kde', plot_kws={'alpha':0.5, 's':10})
plt.suptitle('Nuages de points des variables quantitatives', y=1.02)
plt.show()


Biais d’âge : Les patients diagnostiqués "Malades" semblent en moyenne plus âgés que ceux avec "No Finding".

Biais de genre : Les hommes et les femmes n’ont pas la même répartition des diagnostics. Bien que les rapports malades/sain dans les deux genres semblent équilibrer ( détailler dans la suite )

Biais de genre dans le suivi médical : Différence visible entre hommes et femmes en termes de nombre de suivis mais qui peut être corrigé en corrigeant le biais sur la répartitions des sexes.


In [51]:
# Pour deux variables quantitatives

#  Table de contingence pour deux variables qualitatives
def table_contingence(var1, var2, top_n=1):
    top_categories = df_clean[var2].value_counts().index[:top_n]  # Garder les top_n valeurs les plus fréquentes
    df_filtered = df_clean.copy()
    df_filtered[var2] = df_filtered[var2].apply(lambda x: x if x in top_categories else "Malades")
    
    table = pd.crosstab(df_filtered[var1], df_filtered[var2])
    print("\nTable de contingence :")
    print(table)
    table.plot(kind='bar', stacked=True, figsize=(8, 5))
    plt.title(f"Distribution de {var1} selon {var2}")
    plt.xticks(rotation=45)
    plt.show()
    
    # Ajout du graphique en mosaïque avec correction et lisibilité améliorée
    top_categories = df_clean[var2].value_counts().index[:5]  # Garder les top_n valeurs les plus fréquentes
    df_filtered = df_clean.copy()
    df_filtered[var2] = df_filtered[var2].apply(lambda x: x if x in top_categories else "Malades")
    table = pd.crosstab(df_filtered[var1], df_filtered[var2])
    
    plt.figure(figsize=(10, 6))
    mosaic_data = {(str(k1), str(k2)): v for (k1, k2), v in table.stack().items()}
    mosaic(mosaic_data, title=f"Mosaïque de {var1} et {var2}")
    plt.xticks(rotation=45, fontsize=8)
    plt.yticks(fontsize=8)
    plt.show()

#   Corrélation Pearson et Spearman entre deux variables quantitatives
def correlation_quantitative(var1, var2):
    pearson_corr, _ = pearsonr(df_clean[var1], df_clean[var2])
    spearman_corr, _ = spearmanr(df_clean[var1], df_clean[var2])
    print(f"\nCorrélation de Pearson entre {var1} et {var2} : {pearson_corr:.3f}")
    print(f"Corrélation de Spearman entre {var1} et {var2} : {spearman_corr:.3f}")
    

def correlation_ratio(var_qual, var_quant):
    
    top_categories = df_clean[var_qual].value_counts().index[:1]  # Garder la valeur la plus fréquente
    df_filtered = df_clean.copy()
    df_filtered[var_qual] = df_filtered[var_qual].apply(lambda x: x if x in top_categories else "Malades")
    # Encodage des valeurs qualitatives en numérique
    le = LabelEncoder()
    df_filtered["encoded_qual"] = le.fit_transform(df_filtered[var_qual])

    # Calcul des variances
    classes = df_filtered.groupby("encoded_qual")[var_quant]
    sst = np.var(df_filtered[var_quant]) * len(df_filtered)
    ssb = sum(len(group) * (group.mean() - df_filtered[var_quant].mean())**2 for _, group in classes)

    # Calcul du rapport de corrélation η²
    eta_squared = ssb / sst
    print(f"\nRapport de corrélation η² entre {var_qual} et {var_quant} : {eta_squared:.3f}")
    return eta_squared


# Exécuter les analyses
table_contingence("Patient Gender", "Finding Labels")  # Ex: Genre vs Diagnostic
correlation_quantitative("Patient Age", "Follow-up #")  # Ex: Age vs num du Suivis du patient
for qual in qualitative_vars:
    for quant in quantitative_vars:
        correlation_ratio(qual, quant)



Table de contingence :
Finding Labels  Malades  No Finding
Patient Gender                     
F                 11513       13201
M                 14384       16454

Corrélation de Pearson entre Patient Age et Follow-up # : -0.036
Corrélation de Spearman entre Patient Age et Follow-up # : 0.001

Rapport de corrélation η² entre Finding Labels et Patient ID : 0.002

Rapport de corrélation η² entre Finding Labels et Patient Age : 0.005

Rapport de corrélation η² entre Finding Labels et Follow-up # : 0.035

Rapport de corrélation η² entre Patient Gender et Patient ID : 0.003

Rapport de corrélation η² entre Patient Gender et Patient Age : 0.000

Rapport de corrélation η² entre Patient Gender et Follow-up # : 0.000


D'après le tableau de contigences et selon la distribution du sexe du patient en fonction du diagnostic ci dessus :
Proportion de malades :

F : 
11513/24714​≈46.58%

M : 
14384/30838≈46.66%

Les pourcentages de malades sont relativement similaires entre les genres, ce qui laisse entendre que la répartition malade/pas malade a été faite de manière equitable. Reste à voir si elle été faite de manière équitable pour les differents diagnostics. Pareil, pour le schéma suivant. Cependant, on confirme une surrepresentation du sexe 'M' comme rapporté dans l'analyse univarié.

corrélation de Pearson et de Spearman entre Patient Age et Follow-up # :


Pearson : -0.036 (très faible corrélation négative)

Spearman : 0.001 (quasi inexistante)

→ L'âge du patient n'a pratiquement aucun lien avec le nombre de suivis,  suggèrant une indépendance entre ces variables. Aucun biais particulier ici.

Corrélation η² entre Finding Labels et les autres variables :


Avec Patient ID : 0.002 (aucune relation significative)

Avec Patient Age : 0.005 (très faible)

Avec Follow-up # : 0.035 (légèrement plus élevé, mais toujours faible)

→ Aucun facteur parmi ceux-ci ne semble fortement influencé par la présence ou l'absence d'une maladie à part le suivi où on trouve quand une faible corrélation. Cela suggère que Finding Labels ne dépend pas fortement de l'âge ou du nombre de suivis, mais une analyse plus approfondie serait nécessaire pour détecter des biais subtils. Car dans un contexte médical, le suivi peut avoir un lien avec la présence d'une maladie ( Légére ouverture à ne pas prendre en compte car nous sommes en Fairness et le contexte des choses n'a pas d'interets)

Corrélation η² entre Patient Gender et les autres variables :


Avec Patient ID : 0.003 (aucune relation)

Avec Patient Age : 0.000 (totalement indépendant)

Avec Follow-up # : 0.000 (aucune relation)

→ evident car le sexe ne présente pas de lien avec son age ni son ID. completement independantes.

In [52]:
#Pour une variable quantitative
#et une variable qualitative

#Les boîtes à moustaches

# Création de boxplots pour chaque paire variable qualitative vs quantitative

top_categories = df_clean["Finding Labels"].value_counts().index[:1]  # Garder la valeur la plus fréquente
df_filtered = df_clean.copy()
df_filtered["Finding Labels"] = df_filtered["Finding Labels"].apply(lambda x: x if x in top_categories else "Malades")

def BoxPlot(qual_var, quant_var, df_filtered):
    plt.figure(figsize=(9, 3))
    sns.boxplot(x=qual_var, y=quant_var, data=df_filtered, palette='Set3')
    plt.title(f'Boxplot : {qual_var} vs {quant_var}')
    plt.xticks(rotation=45, ha='right')
    plt.show()
    
BoxPlot("Finding Labels", "Patient Age",df_filtered)
BoxPlot("Patient Gender", "Follow-up #", df_filtered)
BoxPlot("Patient Gender","Patient Age",df_filtered)

On remarque que chez les hommes le nombre de suivis est plus élevé que chez les femmes, chose pouvant être encore apporté au fait que les hommes soient beaucoup plus nombreux.
L'âge médian semble légèrement plus élevé pour les patients malades que pour ceux sans maladie et surtout pour la tranche 40-60 ans.
la derniere boxplot confirme ce qui a été montré plus haut.

In [53]:
import numpy as np
import plotly.graph_objects as go

def plot_histogram_age_by_gender(df, age_col="Patient Age", gender_col="Patient Gender", bins=25):
    fig = go.Figure()

    # (Optionnel) Filtrer les âges aberrants pour un meilleur affichage
    df_filtered = df[df[age_col].between(0, 100)]

    # 1) Calculer les bornes pour toute la distribution
    _, bin_edges = np.histogram(df_filtered[age_col], bins=bins)

    # 2) Boucler sur chaque genre en réutilisant les mêmes bin edges
    for gender in df_filtered[gender_col].unique():
        mask = (df_filtered[gender_col] == gender)
        # Comptage avec les mêmes bin_edges
        counts, _ = np.histogram(df_filtered.loc[mask, age_col], bins=bin_edges)

        # Ajouter les barres pour chaque genre
        fig.add_trace(go.Bar(
            x=bin_edges[:-1],
            y=counts,
            name=f"Genre={gender}",
            # Pour séparer visuellement les barres par genre
            # offsetgroup=str(gender)
        ))

    # 3) Configuration du layout
    fig.update_layout(
        # 'overlay' pour superposer les barres, 'group' pour les côte à côte
        barmode='overlay',
        xaxis_title="Âge du patient",
        yaxis_title="Effectif",
        title="Distribution de l'âge en fonction du genre"
    )
    # Ajuster l'opacité pour mieux voir la superposition
    fig.update_traces(opacity=0.6)

    fig.show()

# Exemple d'utilisation
plot_histogram_age_by_gender(df_clean)


On observe la repartition des âges en fonction du sexe des patients de notre jeu de données. On voit que la repartition des âges est plutôt ressemblante pour les sexes ( Malgré une domination du genre 'M' dans les tranches d'age âgées) On rappelle que la repartition des effectifs n'est pas équilibrée en fonction des genres.
On observe que les hommes ont tendance a être plus vieux.

In [54]:
# Utilisation du dataset nettoyé de base
df_t = df_clean.copy()

# Définir le label binaire : 1 si le patient est malade, 0 sinon.
df_t['malade'] = (df_t['Finding Labels'] != 'No Finding').astype(int)

# Calculer la proportion de malades pour chaque genre
p_f = df_t[df_t['Patient Gender'] == 'F']['malade'].mean()
p_m = df_t[df_t['Patient Gender'] == 'M']['malade'].mean()

# Calculer les métriques de fairness
SPDg = p_f - p_m
DIg = p_f / p_m if p_m != 0 else float('inf')

print("Proportion des femmes malades :", p_f)
print("Proportion des hommes malades :", p_m)
print("(F - M) Statistical Parity Difference     :", SPDg)
print("(F / M) Disparate Impact             :", DIg)

Proportion des femmes malades : 0.46584931617706565
Proportion des hommes malades : 0.4664375121603217
(F - M) Statistical Parity Difference     : -0.0005881959832560457
(F / M) Disparate Impact             : 0.9987389608084225


Les valeurs affichés confirment la bonne répartition des maladies en fonction du genre comme présentées plus haut.

L'analyse bivarié a mis en avant un biais que l'on devrait considerer dans la partie suivante de Pre-processing. On voit bien d'après l'analyse que l'on a plus de chances d'être malade lorsqu'on est vieux ( dans notre cas, je dirais plus de 40ans ) et que l'on est de sexe masculin. chose à corriger

# 4 - Pre-processing

In [55]:
#Suppression des données où l'âge du patient est superieur à 100 ans. Pour corriger le biais sur l'âge
# On a corrigé notre premier biais 

df_t = df_t[df_t['Patient Age'] <= 100]
df_t.describe()


,Follow-up #,Patient ID,Patient Age,Original_Image_Width,Original_Image_Height,PixelSpacingX,PixelSpacingY,malade
count,55549.000000,55549.000000,55549.000000,55549.000000,55549.000000,55549.000000,55549.000000,55549.000000
mean,8.544024,14331.876451,46.902987,2645.624638,2483.619903,0.155607,0.155607,0.466165
std,14.865719,8419.694448,16.505568,340.519054,399.883694,0.016137,0.016137,0.498858
min,0.000000,1.000000,1.000000,1189.000000,966.000000,0.115000,0.115000,0.000000
25%,0.000000,7416.000000,35.000000,2500.000000,2048.000000,0.143000,0.143000,0.000000
50%,3.000000,13993.000000,48.000000,2510.000000,2544.000000,0.143000,0.143000,0.000000
75%,10.000000,20629.000000,59.000000,2992.000000,2991.000000,0.168000,0.168000,1.000000
max,157.000000,30805.000000,95.000000,3550.000000,3166.000000,0.198800,0.198800,1.000000


In [56]:
df_t.head(10)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,Original_Image_Width,Original_Image_Height,PixelSpacingX,PixelSpacingY,malade
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,1
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,1
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,1
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,0
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,1
5,00000003_001.png,Hernia,1,3,74,F,PA,2500,2048,0.168,0.168,1
6,00000003_002.png,Hernia,2,3,75,F,PA,2048,2500,0.168,0.168,1
7,00000003_003.png,Hernia|Infiltration,3,3,76,F,PA,2698,2991,0.143,0.143,1
8,00000003_004.png,Hernia,4,3,77,F,PA,2500,2048,0.168,0.168,1
9,00000003_005.png,Hernia,5,3,78,F,PA,2686,2991,0.143,0.143,1


In [57]:
df_t.shape, df.shape, df_clean.shape

((55549, 12), (55552, 13), (55552, 11))

Au lieu de sous-echantillonner les patients 'M', on va ajouter des poids à chaque donnée afin de corriger les biais identifiés.
Nous allons utiliser des algorithmes proposées dans le cours afin de corriger nos 3 derniers biais :
- les personnes categorisées comme âgées ont plus de chance d'être malade
- Probleme de proportions au niveau du jeu de données : beaucoup plus d'hommes malades que de femmes malades et pareil pour les individus sains
- Beaucoup plus d'individus hommes âgées que de femmes âgées ( ce qui crée une forme d'inequité dans le future apprentissage de notre modèle )

Les algos utilisés prochainement proviennent et ont été montré en cours


In [64]:
# Créer une variable catégorielle pour l'âge
df_t['tranche_age'] = df_t['Patient Age'].apply(lambda x: 'jeune' if x < 45 else 'plus_de_45') # age que j'ai choisi comme frontieres à partir des analyses précédentes


gender_distr = df_t.groupby('tranche_age')['Patient Gender'].value_counts(normalize=True).unstack()

print(gender_distr)

# Pour mesurer la fairness, on peut par exemple comparer la proportion de femmes entre les groupes
prop_f_jeune = gender_distr.loc['jeune', 'F']
prop_f_vieux   = gender_distr.loc['plus_de_45', 'F']

SPD_gender_age = prop_f_jeune - prop_f_vieux
DI_gender_age  = prop_f_jeune / prop_f_vieux if prop_f_vieux != 0 else float('inf')

print("Proportion de femmes dans le groupe 'jeune'           :", prop_f_jeune)
print("Proportion de femmes dans le groupe 'plus_de_45'   :", prop_f_vieux)
print("Statistical Parity Difference (F, jeune - vieux)       :", SPD_gender_age)
print("Disparate Impact (F, jeune / vieux)                    :", DI_gender_age)

group_means = df_t.groupby('tranche_age')['malade'].mean()

prop_jeune = group_means['jeune']
prop_vieux   = group_means['plus_de_45']


SPD = prop_jeune - prop_vieux
# 2) Disparate Impact
if prop_vieux == 0:
    DI = float('inf')
else:
    DI = prop_younger / prop_older

print("Proportion de malades chez les < 45 ans  :", prop_jeune)
print("Proportion de malades chez les >= 45 ans:", prop_vieux)
print("Statistical Parity Difference           :", SPD)
print("Disparate Impact                        :", DI)

Patient Gender         F         M
tranche_age                       
jeune           0.458597  0.541403
plus_de_45      0.435500  0.564500
Proportion de femmes dans le groupe 'jeune'           : 0.4585967549405367
Proportion de femmes dans le groupe 'plus_de_45'   : 0.4354995444883085
Statistical Parity Difference (F, jeune - vieux)       : 0.02309721045222818
Disparate Impact (F, jeune / vieux)                    : 1.0530361299903686
Proportion de malades chez les < 45 ans  : 0.42782616384455546
Proportion de malades chez les >= 45 ans: 0.49249924081384755
Statistical Parity Difference           : -0.06467307696929209
Disparate Impact                        : 0.8686839052534968


In [65]:
l = len(df_t)

group_c = df_t['tranche_age'].value_counts()      
class_c = df_t['malade'].value_counts()

join_c = df_t.groupby(['tranche_age', 'malade']).size()

poids = {}
for x in group_c.index:
    for y in class_c.index:
        joint_count = join_c.get((x, y), 0)
        if joint_count == 0:
            poid = 0
        else:
            poid = (group_c[x] * class_c[y]) / (l * joint_count)
        poids[(x, y)] = poid

df_t['weight'] = df_t.apply(lambda row: poids[(row['tranche_age'], row['malade'])], axis=1)


# Vérification par affichage de la moyenne des poids par groupe d'âge et label
print(df_t.groupby(['tranche_age', 'malade'])['weight'].mean())

tranche_age  malade
jeune        0         0.932994
             1         1.089613
plus_de_45   0         1.051890
             1         0.946529
Name: weight, dtype: float64


Plus de personnes malades au dela de 45, encore plus veridique avec cette métrique.

In [66]:
# Calculer les moyennes pondérées pour la variable 'is_diseased' pour chaque groupe d'âge
weighted_means = df_t.groupby('tranche_age').apply(lambda g: (g['malade'] * g['weight']).sum() / g['weight'].sum())

prop_younger_weighted = weighted_means['jeune']
prop_older_weighted   = weighted_means['plus_de_45']

# Calculer les métriques de fairness pondérées
SPD_weighted = prop_younger_weighted - prop_older_weighted
DI_weighted = prop_younger_weighted / prop_older_weighted if prop_older_weighted != 0 else float('inf')

print("Proportion pondérée de malades pour < 50 ans   :", prop_younger_weighted)
print("Proportion pondérée de malades pour >= 50 ans :", prop_older_weighted)
print("Weighted Statistical Parity Difference       :", SPD_weighted)
print("Weighted Disparate Impact                    :", DI_weighted)

Proportion pondérée de malades pour < 50 ans   : 0.46616500747088147
Proportion pondérée de malades pour >= 50 ans : 0.46616500747088163
Weighted Statistical Parity Difference       : -1.6653345369377348e-16
Weighted Disparate Impact                    : 0.9999999999999997


C:\Users\dewey\AppData\Local\Temp\ipykernel_11340\1933944682.py:2: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



Weighted Disparate Impact                    : 0.9999999999999997 pratiquement à 1, donc le biais est moins impactant, mais pas forcémment disparu.

In [67]:
# Convertir 'Patient Gender' en numérique
df_t['Patient Gender'] = df_t['Patient Gender'].astype('category').cat.codes

# Exclure les colonnes non numériques sauf 'Patient Gender'
df_numeric = df_t.select_dtypes(include=['number']).copy()

# 1. Conversion du DataFrame en BinaryLabelDataset pour AIF360
dataset = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=df_numeric,
    label_names=['malade'],
    protected_attribute_names=['Patient Gender']
)

# 2. Application du Disparate Impact Remover
dir = DisparateImpactRemover(repair_level=1.0)  # 1.0 = suppression maximale du biais
dataset_transformed = dir.fit_transform(dataset)

# 3. Conversion en DataFrame après transformation
df_transformed = dataset_transformed.convert_to_dataframe()[0]

# Vérification de la mitigation
print(df_transformed.groupby('Patient Gender')['malade'].mean())


Patient Gender
0.0    0.465849
1.0    0.466418
Name: malade, dtype: float64


Ici, aussi, le biais lié au sex du patient et au risque de maladie semblent avoir été corrigé.

In [68]:
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
import numpy as np

# Convertir 'Patient Gender' et 'age_group' en numérique
df_t['Patient Gender'] = df_t['Patient Gender'].astype('category').cat.codes
df_t['tranche_age'] = df_t['tranche_age'].astype('category').cat.codes

# Exclure les colonnes non numériques sauf les attributs protégés
df_numeric = df_t.select_dtypes(include=['number']).copy()

# S'assurer que 'age_group' est bien présent
df_numeric['tranche_age'] = df_t['tranche_age']

# 1. Conversion du DataFrame en BinaryLabelDataset pour AIF360
dataset = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=df_numeric,
    label_names=['malade'],
    protected_attribute_names=['Patient Gender', 'tranche_age']
)

# 2. Application du Reweighting (biais âge)
reweigher = Reweighing(
    unprivileged_groups=[{'Patient Gender': 0}],
    privileged_groups=[{'Patient Gender': 1}]
)
dataset_reweighted = reweigher.fit_transform(dataset)



np.random.seed(42)
num_changes = int(0.05 * len(df_t))  # Modifier 5% des données
mask_favored = (df_t['Patient Gender'] == 1) & (df_t['malade'] == 1)
mask_unfavored = (df_t['Patient Gender'] == 0) & (df_t['malade'] == 0)

favored_indices = df_t[mask_favored].sample(n=num_changes, random_state=42).index
unfavored_indices = df_t[mask_unfavored].sample(n=num_changes, random_state=42).index

df_t.loc[favored_indices, 'malade'] = 0  # Réduire l'avantage du groupe favorisé
df_t.loc[unfavored_indices, 'malade'] = 1  # Augmenter les chances du groupe défavorisé

# 4. Conversion en DataFrame après transformation
df_transformed = dataset_reweighted.convert_to_dataframe()[0]


# Vérification de la mitigation
print(df_transformed.groupby(['Patient Gender', 'tranche_age'])['malade'].mean())


Patient Gender  tranche_age
0.0             0.0            0.417333
                1.0            0.500941
1.0             0.0            0.436714
                1.0            0.485986
Name: malade, dtype: float64


Le bias lié entre "Patient Gender" et "Age Group" semblent avoir été corrigées comme le montre les metrics qui temoignent d'un certain equilibre

In [32]:
df_transformed.to_csv('data/Ismael-wane-corrigé.csv', index=False)

# Conclusion

Le but de notre projet était de corriger des biais que l'on aurait détécté dans les etapes d'analyse. 

Pour se faire, on a tout d'abord commencer par préparer et nettoyer notre jeu de données : pour se faire, nous avons chargé les données puis observer les differentes données que nous avions à disposition ainsi que leur type. Nous avons aussi dans cette meme partie, retirer la colonne "Unnamed 11: " et renommer nos colonnes pour une meilleur visibilité.

Ensuite, nous avons commencé l'analyse descriptive de nos données, nous avons commencé par effectuer l'analyse univarié et enfin, l'analyse bivarié, tout cela dans le but de reperer de potentiels biais.

Puis, nous avons, à partir des biais détectés dans l'étape précedente, effectué un pre-processing de nos données dans l'objectif de corrigé nos biais et ainsi générer un nouveau jeu de données corrigeant ces biais. 
